#### On cherche à garder les meilleures variables pour notre modèle de classification :
- On va donc utiliser les features importances de randomforest et puis ne l'entraîner que sur les meilleures features pour que le modèle ne soit pas trop lourd
- **Un enjeu clef est de voir si les données de l'open data socio-économique de l'INSEE joue sur les étiquettes DPE**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
from pre_processing import preprocess_data, import_data
import numpy as np

In [ ]:
df = import_data('data/merged_69.csv') # Si vous voulez re-executer le code, il faut changer le chemin du fichier par ../data/dataset_M2_enedis.csv
print(df.shape)
# Entrée ayant la date réception la plus petite et la plus grande
# On recherche les dates
print(df['Date_réception_DPE'].min())
print(df['Date_réception_DPE'].max())
print(df['Date_réception_DPE'])


d:\M2\Python\Projet\pre_processing.py:15: DtypeWarning: Columns (33,35,38,51,57,69,72,99,115) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(chemin, sep=';')


Import réussi
(287165, 131)
2021-07-01
2024-09-16
0         2024-08-30
1         2024-07-25
2         2024-08-13
3         2024-07-30
4         2024-09-11
             ...    
287160    2024-06-27
287161    2024-03-01
287162    2024-03-04
287163    2024-06-24
287164    2024-02-29
Name: Date_réception_DPE, Length: 287165, dtype: object


In [ ]:
df = preprocess_data(df, sauvegarde=False)

0
1138
0         1121
1         1085
2         1104
3         1090
4         1133
          ... 
287160    1059
287161     960
287162     961
287163    1056
287164     959
Name: Date_réception_DPE, Length: 287165, dtype: int32


In [21]:
df.head()
print(df.info()) # Voir le type des colonnes : ne devrait avoir que des float64 car Encoding des variables catégorielles
# Définition des variables explicatives et de la variable cible
target = "Etiquette_DPE"
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287165 entries, 0 to 287164
Columns: 103 entries, Ubat_W/m²_K to Complément_d'adresse_logement
dtypes: float64(103)
memory usage: 225.7 MB
None


In [ ]:
RF_model = RandomForestClassifier(n_estimators=100, random_state=42)
print("Entrainement du modèle...")
RF_model.fit(X_train, y_train)
y_pred = RF_model.predict(X_test)


Entrainement du modèle...


In [ ]:
print("Résultats du modèle...")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Meilleures features...")
feature_importances = RF_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]
top_n = 5  # Limite à 10 -> avec 10 features, on a déjà 0.991 d'accuracy, avec 5 features, on a 0.987
top_features = X.columns[indices[:top_n]]
top_importances = feature_importances[indices[:top_n]]
for i, feature in enumerate(top_features):
  print(f"{i+1}. {feature}: {top_importances[i]}")

print(f"Si on doit réduire la dimensions, il ne faudra garde que les {top_n} features ci-dessus")

Résultats du modèle...
Accuracy: 0.9949390597794544
Meilleures features...
1. Conso_5_usages_par_m²_é_primaire: 0.23084911045301518
2. Conso_5_usages/m²_é_finale: 0.117427696805111
3. Emission_GES_5_usages_par_m²: 0.10900606994831526
4. Etiquette_GES: 0.09830970376943249
5. Coût_éclairage: 0.02219566954635474
Si on doit réduire la dimensions, il ne faudra garde que les 5 features ci-dessus


**Essayons de réentraîner le modèle avec ces 10 features et regardons si l'accuracy à changer, elle est égal à 99%, ce qui est déjà très haut**

In [ ]:
RF_model_alleger = RandomForestClassifier(n_estimators=100, random_state=42)
print("Entrainement du modèle...")
X_train, X_test = X_train[top_features], X_test[top_features]
RF_model_alleger.fit(X_train, y_train)
y_pred = RF_model_alleger.predict(X_test)

print("Résultats du modèle...")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Entrainement du modèle...
Résultats du modèle...
Accuracy: 0.9867208357515961


**On voit que le modèle garde une accuracy très élevé (99%), même en ne retenant que les 10 premières variables, essayons avec 5**

-> Nous avons 0.987, on va donc garder un modèle avec les 5 meilleures features, essayons aussi de baisser le nombre d'estimator


In [30]:
RF_model_alleger = RandomForestClassifier(n_estimators=10, random_state=42)
print("Entrainement du modèle...")
X_train, X_test = X_train[top_features], X_test[top_features]
RF_model_alleger.fit(X_train, y_train)
y_pred = RF_model_alleger.predict(X_test)

print("Résultats du modèle...")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Entrainement du modèle...
Résultats du modèle...
Accuracy: 0.9867208357515961


**Lorsqu'on passe de 100 à 10 estimator, l'accuracy ne change pas, on va donc en garder que 10**

### Export du modèle

In [32]:

try:
  with open('RF_alleger_Classification_Etiquette.pkl', 'wb') as file:
      pickle.dump(RF_model_alleger, file)
  print(f"Modèle sauvegardé avec succès sous : {'RF_alleger_Classification_Etiquette.pkl'}")
except Exception as e:
  print(f"Erreur lors de la sauvegarde du modèle : {e}")

Modèle sauvegardé avec succès sous : RF_alleger_Classification_Etiquette.pkl


In [34]:
list(top_features)

['Conso_5_usages_par_m²_é_primaire',
 'Conso_5_usages/m²_é_finale',
 'Emission_GES_5_usages_par_m²',
 'Etiquette_GES',
 'Coût_éclairage']